# Cleaning

You would probably like to live in a place where your car won't be jacked or your packages stolen. Can we 
predict which counties will have low property crime in 2019 based on 2018 data?

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import math
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

## Data Sources

Crime Data:

https://ucr.fbi.gov/crime-in-the-u.s/2018/crime-in-the-u.s.-2018/tables/table-10/table-10.xls/view

https://ucr.fbi.gov/crime-in-the-u.s/2019/crime-in-the-u.s.-2019/tables/table-10/table-10.xls/view

https://ucr.fbi.gov/crime-in-the-u.s/2018/crime-in-the-u.s.-2018/tables/table-10/table-10-data-declaration

Population Data:

https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html#par_textimage_739801612

Income & Employment Data:

https://apps.bea.gov/iTable/iTable.cfm?reqid=70&step=1&acrdn=6

https://apps.bea.gov/regional/histdata/releases/1120lapi/LAPI-Methodology.pdf

Average Temperature & Precipitation:

https://www.ncdc.noaa.gov/cag/county/mapping/110/tavg/202012/12/anomaly

Demographic Data:

https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/cc-est2019-alldata.pdf

https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-detail.html

## Crime Data

In [ ]:
crime2018_df = pd.read_csv('Table_10_Offenses_Known_to_Law_Enforcement_by_State_by_Metropolitan_and_Nonmetropolitan_Counties_2018.csv', thousands=',')

In [ ]:
crime2018_df.head()

The original Excel file uses merged cells for the State, so I can use fillna with ffill.

In [ ]:
crime2018_df[['State']] = crime2018_df[['State']].fillna(method='ffill')

In [ ]:
crime2018_df.isna().sum()

I'm interested in predicting Property Crime.

In [ ]:
crime2018_df.drop(crime2018_df[crime2018_df['Property crime'].isna()].index, inplace=True)

In [ ]:
crime2018_df.isna().sum()

In [ ]:
crime_drop_cols = [col for col in crime2018_df.columns.to_list() if col not in ['State','County','Property crime']]
crime2018_df.drop(columns=crime_drop_cols, inplace=True)

In [ ]:
crime2018_df.head()

In [ ]:
crime2019_df = pd.read_csv('Table_10_Offenses_Known_to_Law_Enforcement_by_State_by_Metropolitan_and_Nonmetropolitan_Counties_2019.csv', thousands=',')

In [ ]:
crime2019_df[['State']] = crime2019_df[['State']].fillna(method='ffill')

In [ ]:
crime2019_df.drop(columns=crime_drop_cols, inplace=True)
crime2019_df.isna().sum()

In [ ]:
crime2019_df.drop(crime2019_df[crime2019_df['Property crime'].isna()].index, inplace=True)

In [ ]:
crime2019_df.isna().sum()

In [ ]:
crime2019_df

In [ ]:
crime2018_standard_df = crime2018_df.copy().reset_index(drop=True)
crime2018_standard_df['State'] = pd.Series([state.split(' - ')[0] for state in crime2018_df['State'].to_list()]).str.title()
crime2018_standard_df

In [ ]:
crime2019_standard_df = crime2019_df.copy().reset_index(drop=True)
crime2019_standard_df['State'] = pd.Series([state.split(' - ')[0] for state in crime2019_df['State'].to_list()]).str.title()
crime2019_standard_df

In [ ]:
[county for county in crime2018_standard_df['County'].to_list() if ' ' in county]

In [ ]:
crime2018_standard_df['County'] = crime2018_standard_df['County'].apply(lambda x: x.split(' County')[0].split(' Police')[0].split(' Public')[0])
crime2018_standard_df['County'] = crime2018_standard_df['County'].apply(lambda x: x.split('3')[0].split('5')[0])
crime2019_standard_df['County'] = crime2019_standard_df['County'].apply(lambda x: x.split(' County')[0].split(' Police')[0].split(' Public')[0])
crime2019_standard_df['County'] = crime2019_standard_df['County'].apply(lambda x: x.split('3')[0].split('5')[0])

In [ ]:
crime2018_standard_df[crime2018_standard_df.duplicated(subset=['State','County'], keep=False)]

In [ ]:
crime2018_standard_df = crime2018_standard_df.groupby(by=['State','County']).sum().reset_index()
crime2019_standard_df = crime2019_standard_df.groupby(by=['State','County']).sum().reset_index()

## Temperature and Precipitation Data

In [ ]:
ave_temp_2018_df = pd.read_csv('tavg-2018.csv')
ave_temp_2019_df = pd.read_csv('tavg-2019.csv')
precip_2018_df = pd.read_csv('pcp-2018.csv')
precip_2019_df = pd.read_csv('pcp-2019.csv')

In [ ]:
ave_temp_2018_df.head()

In [ ]:
ave_temp_2018_df.drop(columns=['Rank','Anomaly (1901-2000 base period)','1901-2000 Mean'], inplace=True)
ave_temp_2019_df.drop(columns=['Rank','Anomaly (1901-2000 base period)','1901-2000 Mean'], inplace=True)
ave_temp_2018_df.rename(columns={'Value':'Annual Average Temp'}, inplace=True)
ave_temp_2019_df.rename(columns={'Value':'Annual Average Temp'}, inplace=True)

In [ ]:
ave_temp_2018_df.head()

In [ ]:
precip_2018_df.head()

In [ ]:
precip_2018_df.drop(columns=['Rank','Anomaly (1901-2000 base period)','1901-2000 Mean'], inplace=True)
precip_2019_df.drop(columns=['Rank','Anomaly (1901-2000 base period)','1901-2000 Mean'], inplace=True)
precip_2018_df.rename(columns={'Value':'Total Annual Precipitation'}, inplace=True)
precip_2019_df.rename(columns={'Value':'Total Annual Precipitation'}, inplace=True)

In [ ]:
precip_2018_df.head()

In [ ]:
temp_precip_2018_df = pd.merge(ave_temp_2018_df, precip_2018_df)
temp_precip_2019_df = pd.merge(ave_temp_2019_df, precip_2019_df)

In [ ]:
temp_precip_2018_df.head()

In [ ]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [ ]:
temp_precip_2018_df['State'] = temp_precip_2018_df['Location ID'].apply(lambda x: x.split('-')[0]).map(abbrev_to_us_state)

In [ ]:
temp_precip_2018_df['County'] = temp_precip_2018_df['Location'].apply(lambda x: x.split(' County')[0])

In [ ]:
temp_precip_2018_df.drop(columns=['Location ID', 'Location'], inplace=True)

In [ ]:
temp_precip_2019_df['State'] = temp_precip_2019_df['Location ID'].apply(lambda x: x.split('-')[0]).map(abbrev_to_us_state)
temp_precip_2019_df['County'] = temp_precip_2019_df['Location'].apply(lambda x: x.split(' County')[0])
temp_precip_2019_df.drop(columns=['Location ID', 'Location'], inplace=True)

## Income and Employment Data

In [ ]:
income_df = pd.read_csv('inc_emp_2018-2019.csv')

In [ ]:
income_df.head(10)

In [ ]:
income_df.drop(columns='LineCode', inplace=True)
income_df.replace({'Per capita personal income (dollars) 4/':'Per capita personal income ($)'}, inplace=True)
per_cap_inc_df = income_df[income_df['Description'] == 'Per capita personal income ($)'].copy()
per_cap_inc_df.rename(columns={'2018':'Per capita personal income ($) 2018','2019':'Per capita personal income ($) 2019'}, inplace=True)
per_cap_inc_df.drop(columns='Description', inplace=True)

In [ ]:
per_cap_inc_df.head(5)

In [ ]:
employment_df = income_df[income_df['Description'] == 'Total employment'].copy()
employment_df.rename(columns={'2018':'Total employment 2018','2019':'Total employment 2019'}, inplace=True)
employment_df.drop(columns='Description', inplace=True)
employment_df.head(5)

In [ ]:
income_df.replace({'Population (persons) 3/':'Population'}, inplace=True)
population_df = income_df[income_df['Description'] == 'Population'].copy()
population_df.rename(columns={'2018':'Population 2018','2019':'Population 2019'}, inplace=True)
population_df.drop(columns='Description', inplace=True)
population_df.head(5)

In [ ]:
inc_emp_df = pd.merge(per_cap_inc_df, employment_df)
inc_emp_pop_2018_df = pd.merge(population_df, inc_emp_df)
inc_emp_pop_2019_df = inc_emp_pop_2018_df.copy()
collist = inc_emp_pop_2018_df.columns.to_list()
collist2018 = [col for col in collist if '2019' not in col]
collist2019 = [col for col in collist if '2018' not in col]
inc_emp_pop_2018_df = inc_emp_pop_2018_df[collist2018]
inc_emp_pop_2019_df = inc_emp_pop_2019_df[collist2019]

In [ ]:
inc_emp_pop_2018_df.drop(index=inc_emp_pop_2018_df[inc_emp_pop_2018_df['Population 2018'] == '(NA)'].index, inplace=True)
inc_emp_pop_2019_df.drop(index=inc_emp_pop_2019_df[inc_emp_pop_2019_df['Population 2019'] == '(NA)'].index, inplace=True)

In [ ]:
inc_emp_pop_2018_df['Population 2018'] = inc_emp_pop_2018_df['Population 2018'].astype(int)
inc_emp_pop_2019_df['Population 2019'] = inc_emp_pop_2019_df['Population 2019'].astype(int)

In [ ]:
inc_emp_pop_2018_df['Per capita personal income ($)'] = inc_emp_pop_2018_df['Per capita personal income ($) 2018'].astype(int)
inc_emp_pop_2019_df['Per capita personal income ($)'] = inc_emp_pop_2019_df['Per capita personal income ($) 2019'].astype(int)

In [ ]:
inc_emp_pop_2018_df['Total employment'] = inc_emp_pop_2018_df['Total employment 2018'].astype(int)
inc_emp_pop_2019_df['Total employment'] = inc_emp_pop_2019_df['Total employment 2019'].astype(int)

In [ ]:
inc_emp_pop_2018_df.drop(columns=['Per capita personal income ($) 2018','Total employment 2018'], inplace=True)
inc_emp_pop_2019_df.drop(columns=['Per capita personal income ($) 2019','Total employment 2019'], inplace=True)

In [ ]:
inc_emp_pop_2018_df[inc_emp_pop_2018_df['GeoName'].apply(lambda x: len(x.split(','))) > 2]

The county name appears first, the city names combined with this county appear next.

In [ ]:
inc_emp_pop_2018_df['County'] = inc_emp_pop_2018_df['GeoName'].apply(lambda x: x.split(',')[0])
inc_emp_pop_2018_df['ST'] = inc_emp_pop_2018_df['GeoName'].apply(lambda x: x.split(',')[-1])

In [ ]:
inc_emp_pop_2018_df.head()

In [ ]:
inc_emp_pop_2018_df['ST'].unique()

In [ ]:
inc_emp_pop_2018_df['ST'] = inc_emp_pop_2018_df['ST'].apply(lambda x: x.split('*')[0].strip())

In [ ]:
inc_emp_pop_2018_df['State'] = inc_emp_pop_2018_df['ST'].map(abbrev_to_us_state)

In [ ]:
inc_emp_pop_2018_df.drop(columns=['GeoName','ST'], inplace=True)

In [ ]:
inc_emp_pop_2019_df['County'] = inc_emp_pop_2019_df['GeoName'].apply(lambda x: x.split(',')[0])
inc_emp_pop_2019_df['ST'] = inc_emp_pop_2019_df['GeoName'].apply(lambda x: x.split(',')[-1])
inc_emp_pop_2019_df['ST'] = inc_emp_pop_2019_df['ST'].apply(lambda x: x.split('*')[0].strip())
inc_emp_pop_2019_df['State'] = inc_emp_pop_2019_df['ST'].map(abbrev_to_us_state)
inc_emp_pop_2019_df.drop(columns=['GeoName','ST'], inplace=True)

## Demographic Data

In [ ]:
demographics_df = pd.read_csv('cc-est2018-2019.csv')

In [ ]:
demographics_df.head()

Year 11 is 2018, 12 is 2019.

AGEGRP is age // 5 + 1, while 0 represents the total of all age groups.

In [ ]:
demographics_df.describe()

In [ ]:
len(demographics_df.columns)

Over 40 columns for each racial/ethnic group, 2 years, and 18 age groups. That would make for a lot of columns in a pivot table. I will drop some of smaller-valued ethnic columns.

In [ ]:
collist = demographics_df.columns.to_list()
hlist = [col for col in collist if 'H' in col]
hlist.remove('H_MALE')
hlist.remove('H_FEMALE')
demographics_df.drop(columns=hlist, inplace=True)

In [ ]:
demographics_df.head()

In [ ]:
demo2018_df = demographics_df[demographics_df['YEAR'] == 11].copy()
demo2019_df = demographics_df[demographics_df['YEAR'] == 12].copy()
demo2018_df.drop(columns='YEAR', inplace=True)
demo2019_df.drop(columns='YEAR', inplace=True)

In [ ]:
demo2018_df.head()

In [ ]:
demo2018pivot_df = demo2018_df.pivot(index=['STNAME','CTYNAME'], columns='AGEGRP')

In [ ]:
demo2018pivot_df.head()

In [ ]:
demo2018pivot_df.drop(columns=['STATE','COUNTY'], inplace=True)

In [ ]:
demo2018pivot_df.head()

In [ ]:
collist = demo2018pivot_df.columns.to_list()
level1 = [col[0] for col in collist]
level2 = [col[1] for col in collist]
collist1 = list(set(level1))
collist2 = list(set(level2))

In [ ]:
indlist = demo2018pivot_df.index.to_list()
ind1 = [i[0] for i in indlist]
ind2 = [i[1] for i in indlist]
demo_wide_2018_df = pd.DataFrame()
demo_wide_2018_df['State'] = pd.Series(ind1)
demo_wide_2018_df['County'] = pd.Series(ind2)
for race in collist1:
  for age_group in collist2:
    demo_wide_2018_df[f'{race}_{age_group}'] = demo2018pivot_df[race,age_group].values

In [ ]:
collist = demo_wide_2018_df.columns.to_list()
collist.remove('State')
collist.remove('County')
collist.remove('TOT_POP_0')
for col in collist:
  demo_wide_2018_df[f'FRAC_{col}'] = demo_wide_2018_df[col]/demo_wide_2018_df['TOT_POP_0']
  demo_wide_2018_df.drop(columns=col, inplace=True)

In [ ]:
demo_wide_2018_df.head(5)

In [ ]:
demo2019pivot_df = demo2019_df.pivot(index=['STNAME','CTYNAME'], columns='AGEGRP')
demo2019pivot_df.drop(columns=['STATE','COUNTY'], inplace=True)
collist = demo2019pivot_df.columns.to_list()
level1 = [col[0] for col in collist]
level2 = [col[1] for col in collist]
collist1 = list(set(level1))
collist2 = list(set(level2))
indlist = demo2019pivot_df.index.to_list()
ind1 = [i[0] for i in indlist]
ind2 = [i[1] for i in indlist]
demo_wide_2019_df = pd.DataFrame()
demo_wide_2019_df['State'] = pd.Series(ind1)
demo_wide_2019_df['County'] = pd.Series(ind2)
for race in collist1:
  for age_group in collist2:
    demo_wide_2019_df[f'{race}_{age_group}'] = demo2019pivot_df[race,age_group].values
collist = demo_wide_2019_df.columns.to_list()
collist.remove('State')
collist.remove('County')
collist.remove('TOT_POP_0')
for col in collist:
  demo_wide_2019_df[f'FRAC_{col}'] = demo_wide_2019_df[col]/demo_wide_2019_df['TOT_POP_0']
  demo_wide_2019_df.drop(columns=col, inplace=True)

In [ ]:
multi_word_counties =[county for county in demo_wide_2018_df['County'].to_list() if len(county.split(' ')) > 2 and 'County' in county]
multi_words = [county.split(' County')[0] for county in multi_word_counties]

In [ ]:
demo_wide_2018_df['County'] = demo_wide_2018_df['County'].apply(lambda x: x.split(' County')[0])

In [ ]:
multi_word_counties =[county for county in demo_wide_2019_df['County'].to_list() if len(county.split(' ')) > 2 and 'County' in county]
multi_words = [county.split(' County')[0] for county in multi_word_counties]
demo_wide_2019_df['County'] = demo_wide_2019_df['County'].apply(lambda x: x.split(' County')[0])

## 2018 Combination

In [ ]:
crime_demo_2018_df = pd.merge(crime2018_standard_df, demo_wide_2018_df)
inc_temp_2018_df = pd.merge(inc_emp_pop_2018_df, temp_precip_2018_df)
county_data_2018_df = pd.merge(crime_demo_2018_df, inc_temp_2018_df)

In [ ]:
county_data_2018_df[['Population 2018','TOT_POP_0']].head()

For any further calculation involving the total population of the county, I will use the average of the two total population columns. They should agree, as they are apparently both based on Census estimates, but they don't match exactly but they aren't different enough to justify the time required to track down an answer.

In [ ]:
county_data_2018_df['Population'] = (county_data_2018_df['Population 2018'] + county_data_2018_df['TOT_POP_0']) / 2
county_data_2018_df.drop(columns=['Population 2018','TOT_POP_0'], inplace=True)

In [ ]:
county_data_2018_df['Employment Rate'] = county_data_2018_df['Total employment'] / county_data_2018_df['Population']
county_data_2018_df.drop(columns='Total employment', inplace=True)

In [ ]:
county_data_2018_df['Property Crime Rate'] = county_data_2018_df['Property crime'] / county_data_2018_df['Population']
county_data_2018_df.drop(columns='Property crime', inplace=True)

In [ ]:
id_cols = ['State', 'County', 'GeoFips']
targets = ['Property Crime Rate']
non_X_cols = id_cols + targets
all_X_cols = [col for col in county_data_2018_df.columns.to_list() if col not in non_X_cols]
all_demo_cols = [col for col in all_X_cols if '_' in col]
all_age_cols = [col for col in all_X_cols if 'TOT' in col]
all_race_cols = [col for col in all_demo_cols if col not in all_age_cols]
no_race_X_cols = [col for col in all_X_cols if col not in all_race_cols]
no_demo_X_cols = [col for col in all_X_cols if col not in all_demo_cols]
no_race_cols = [col for col in county_data_2018_df.columns.to_list() if col not in all_race_cols]

In [ ]:
plt.rc('axes', titlesize=10)
plt.rc('axes', labelsize=10)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
sns.pairplot(county_data_2018_df[no_demo_X_cols+targets])

I might have a little more cleaning to do. There are some outliers that might be hurting the accuracy, and I didn't expect the employment rate to go above 1.

The documentation states that part-time and full-time employment are treated equally, so if a lot of people have multiple part-time jobs, or work in a different county from where they live, that can skew this metric.

I will just winsorize the employment rate to 1, because I don't have the means to account for mismatched employment data.

In [ ]:
def limit_to_1(val):
  if val < 1:
    return val
  else:
    return 1

In [ ]:
county_data_2018_df.drop(index=county_data_2018_df[county_data_2018_df['Population'] > 0.5e7].index, inplace=True)
county_data_2018_df.drop(index=county_data_2018_df[county_data_2018_df['Property Crime Rate'] > 0.04].index, inplace=True)
county_data_2018_df['Employment Rate'] = county_data_2018_df['Employment Rate'].apply(limit_to_1)

## 2019 Combination

In [ ]:
crime_demo_2019_df = pd.merge(crime2019_standard_df, demo_wide_2019_df)
inc_temp_2019_df = pd.merge(inc_emp_pop_2019_df, temp_precip_2019_df)
county_data_2019_df = pd.merge(crime_demo_2019_df, inc_temp_2019_df)

In [ ]:
county_data_2019_df['Population'] = (county_data_2019_df['Population 2019'] + county_data_2019_df['TOT_POP_0']) / 2
county_data_2019_df.drop(columns=['Population 2019','TOT_POP_0'], inplace=True)

In [ ]:
county_data_2019_df['Employment Rate'] = county_data_2019_df['Total employment'] / county_data_2019_df['Population']
county_data_2019_df.drop(columns='Total employment', inplace=True)

In [ ]:
county_data_2019_df['Property Crime Rate'] = county_data_2019_df['Property crime'] / county_data_2019_df['Population']
county_data_2019_df.drop(columns='Property crime', inplace=True)

In [ ]:
county_data_2019_df.drop(index=county_data_2019_df[county_data_2019_df['Population'] > 0.5e7].index, inplace=True)
county_data_2019_df.drop(index=county_data_2019_df[county_data_2019_df['Property Crime Rate'] > 0.04].index, inplace=True)
county_data_2019_df['Employment Rate'] = county_data_2019_df['Employment Rate'].apply(limit_to_1)

In [ ]:
county_data_2019_df.head()

# Correlations

Warning: Any model trained with data that includes race or racial characteristics is likely to have racial bias. When racially biased models are used to make decisions in the real world this tends to have negative and unfair consequences for real people. 

The models presented here are only designed to predict crime rates on a county level, and should not be applied to individuals, nor are they recommended for making decisions. They are provided as an example of how models can fit to features which are correlated with the target, but not fundamentally so. This situation results in a potentially harmful bias which is circumstantially useful, but does not generalize well.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(county_data_2018_df.corr())

In [ ]:
def find_max_in_df(df): # find the maximum value in the dataframe and it's column and row
  maxval = 0
  maxrow = ''
  maxcol = ''
  for col in df.columns:
    row = df[col].idxmax()
    if not isinstance(row, str):
      continue
    val = df[col][row]
    if val > maxval:
      maxval = val
      maxrow = row
      maxcol = col
  return maxrow, maxcol, maxval

def weaker_corr_with_target(df, feat1, feat2, target): # choose the feature which has a weaker correlation with the target
  corr1 = df[target][feat1]
  corr2 = df[target][feat2]
  if abs(corr1) < abs(corr2):
    return feat1
  else:
    return feat2

def drop_high_corr(full_df, target, corr_limit=0.95): # generate a list of feature to drop
  corr_matrix = full_df.corr().abs() # which have the highest correlations with other features
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
  maxval = 1.0 # but the lowest correlation with the target.
  to_drop = []
  while maxval > corr_limit:
    feat1, feat2, maxval = find_max_in_df(upper)
    if maxval > corr_limit:
      weak_feat = weaker_corr_with_target(upper, feat1, feat2, target)
      to_drop.append(weak_feat)
      if weak_feat in upper.columns:
        upper.drop(columns=weak_feat, inplace=True)
        upper.drop(index=weak_feat, inplace=True)
    else:
      break
  reduced_df = full_df.drop(columns=to_drop)
  return reduced_df

In [ ]:
county_reduced_2018_df = drop_high_corr(county_data_2018_df, 'Property Crime Rate')#county_data_2018_df.drop(columns=to_drop)

county_no_race_2018_df = drop_high_corr(county_data_2018_df[no_race_cols], 'Property Crime Rate')#county_data_2018_df[no_race_cols].drop(columns=to_drop)

no_race_reduced_X_cols = [col for col in county_no_race_2018_df.columns.to_list() if col not in non_X_cols]
print(len(no_race_reduced_X_cols))
reduced_X_cols = [col for col in county_reduced_2018_df.columns.to_list() if col not in non_X_cols]
print(len(reduced_X_cols))

In [ ]:
county_reduced_2018_df.corr()['Property Crime Rate'].sort_values(ascending=False).head(6)

I wondered if average temperature might be correlated, but I didn't expect it to have the highest correlation (ignoring Violent Crime Rate).

In [ ]:
county_reduced_2018_df.corr()['Property Crime Rate'].sort_values().head(5)

Average temperature and some demographic groups are weakly correlated with crime rates. If we look at negative correlations, young, white females stand out as being less likely to live in high-crime counties.

# No Demographic Data

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.pipeline import make_pipeline

Here I bin the target (Property Crime Rate) into low, moderate, and high, since this is much easier to predict and I don't actually care about the exact number.

In [ ]:
pcr2018 = county_reduced_2018_df['Property Crime Rate']
q1 = pcr2018.quantile(q=0.33)
q2 = pcr2018.quantile(q=0.67)
def crime_level(rate, q1, q2):
  if rate <= q1:
    return 'low'
  elif rate < q2:
    return 'moderate'
  else:
    return 'high'

y = pcr2018.apply(lambda x: crime_level(x, q1, q2))

In [ ]:
def cross_val_avg(model, X, y, n=1):
  clf = make_pipeline(StandardScaler(), model)
  total = 0
  for i in range(n):
    score = cross_val_score(clf, X, y, cv=model_selection.StratifiedKFold(shuffle=True), n_jobs=-1).mean()
    total += score
  return total/n

In [ ]:
X_no_demo = county_reduced_2018_df[no_demo_X_cols]
print(len(X_no_demo.columns))
cross_val_avg(LogisticRegression(), X_no_demo, y, n=5)

In [ ]:
cross_val_avg(RandomForestClassifier(), X_no_demo, y, n=5)

In [ ]:
cross_val_avg(GradientBoostingClassifier(), X_no_demo, y, n=5)

In [ ]:
cross_val_avg(svm.SVC(), X_no_demo, y, n=5)

In [ ]:
no_demo = [[0,0.4878],[3,0.5199],[6,0.514],[1,0.5063]]
no_demo_df = pd.DataFrame(no_demo, columns=['time (s)','score'], index=['LR','RFC','GBC','SVC'])

# No Racial Data

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
X_no_race = county_no_race_2018_df[no_race_reduced_X_cols]
print(len(no_race_X_cols))
cross_val_avg(LogisticRegression(), X_no_race, y, n=5)

In [ ]:
cross_val_avg(RandomForestClassifier(), X_no_race, y, n=5)

In [ ]:
cross_val_avg(GradientBoostingClassifier(), X_no_race, y, n=5)

In [ ]:
cross_val_avg(svm.SVC(), X_no_race, y, n=5)

In [ ]:
no_race = [[0,0.5025],[2,0.5292],[36,0.5256],[1,0.5225]]
no_race_df = pd.DataFrame(no_race, columns=['time (s)','score'], index=['LR','RFC','GBC','SVC'])

# With Racial Data

In [ ]:
print(len(reduced_X_cols))
X_all = county_reduced_2018_df[reduced_X_cols]
cross_val_avg(LogisticRegression(), X_all, y, n=5)

In [ ]:
cross_val_avg(RandomForestClassifier(), X_all, y, n=5)

In [ ]:
cross_val_avg(GradientBoostingClassifier(), X_all, y, n=5)

In [ ]:
cross_val_avg(svm.SVC(), X_all, y, n=5)

In [ ]:
all_features = [[0,0.5009],[3,0.5429],[159,0.5408],[3,0.5278]]
all_features_df = pd.DataFrame(all_features, columns=['time (s)','score'], index=['LR','RFC','GBC','SVC'])

The performance appears to improve only slightly as we add demographic features.

Gradient boosting consistently outperforms random forest, but takes much longer than random forest as the number of features increases. 

The support vector classifier matches the random forest for speed but beats both in mean score.

In [ ]:
plt.rc('axes', titlesize=16)
plt.rc('axes', labelsize=16)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
sp = sns.scatterplot(data=no_demo_df, x='time (s)', y='score', hue=no_demo_df.index, s=70)
sp.legend(bbox_to_anchor= (1.2,1))
sp.set_title('No Demographic Data')

In [ ]:
sp = sns.scatterplot(data=no_race_df, x='time (s)', y='score', hue=no_race_df.index, s=70)
sp.legend(bbox_to_anchor= (1.2,1))
sp.set_title('No Race Data')

In [ ]:
all_features_df.sort_index(inplace=True)
sp = sns.scatterplot(data=all_features_df, x='time (s)', y='score', hue=all_features_df.index, s=70)
sp.legend(bbox_to_anchor= (1.2,1))
sp.set_title('All Features')

# No Race Data, Logistic Regression tuning

In [ ]:
def optimize_param(X, y, model, param_vals, n=5, scale=False):
  param, vals = list(param_vals.items())[0]
  pairs = []
  for val in vals:
    total_score = 0
    if scale:
      method = make_pipeline(StandardScaler(), model(**{param:val}))
    else:
      method = model(**{param:val})
    for i in range(n):
      score = cross_val_score(method, X, y, cv=model_selection.StratifiedKFold(shuffle=True)).mean()
      total_score += score
    mean_score = total_score/n
    pair = [val, mean_score]
    pairs.append(pair)
  x_label = f'{param}'
  y_label = f'Mean Score (n={n})'
  df = pd.DataFrame(data=pairs, columns=[x_label, y_label])
  sns.lineplot(data=df, x=x_label, y=y_label)
  return df

In [ ]:
lr = LogisticRegression
params = {'C':[0.05,0.1,1,2,3,4,5,6]}
C_lr = optimize_param(X_no_race, y, lr, params, 10, scale=True)

In [ ]:
lr = LogisticRegression
params = {'solver':['newton-cg','lbfgs','liblinear','sag','saga']}
C_lr = optimize_param(X_no_race, y, lr, params, n=10, scale=True)

In [ ]:
clf = make_pipeline(StandardScaler(), LogisticRegression())
params = {'solver':['newton-cg','lbfgs','liblinear','sag','saga']}
C_lr = optimize_param(X_no_race, y, lr, params, n=15, scale=True)

In [ ]:
cross_val_avg(LogisticRegression(), X_all, y, n=15)

# No Race Data, Random Forest tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

def grid_search_heatmap(scores, params):
  key_list = list(params[0].keys())
  col_key = key_list[1]
  row_key = key_list[0]
  cols = list(set([par[col_key] for par in params]))
  cols.sort()
  rows = list(set([par[row_key] for par in params]))
  rows.sort()
  df = pd.DataFrame(data=scores.reshape((len(rows),len(cols))), index=rows, columns=cols)
  df = df.sort_index()
  df = df.sort_index(axis=1)
  sns.heatmap(df)

def grid_search_2D(model, params, X, y, n=1):
  scores = []
  for i in range(n):
    gs_clf = GridSearchCV(model, params, cv=model_selection.StratifiedKFold(shuffle=True))
    clf = make_pipeline(StandardScaler(), gs_clf)
    clf.fit(X, y)
    scores.append(gs_clf.cv_results_['mean_test_score'])
  avg_scores = np.mean(scores, axis=0)
  print(avg_scores.max())
  params = gs_clf.cv_results_['params']
  print(params[avg_scores.argmax()])
  grid_search_heatmap(avg_scores, params)

In [ ]:
rfc = RandomForestClassifier
params = {'max_depth':[1,3,5,7,9,11]}
max_depth_odd = optimize_param(X_no_race, y, rfc, params)

In [ ]:
rfc = RandomForestClassifier
params = {'max_depth':[4,5,6,7,8,9,10,11]}
max_depth_4_11 = optimize_param(X_no_race, y, rfc, params)

I will choose a max depth of 9.

In [ ]:
rfc = RandomForestClassifier
params = {'min_samples_split':[2,4,8,16,32,64]}
min_samples_split_pow2 = optimize_param(X_no_race, y, rfc, params)

In [ ]:
rfc = RandomForestClassifier
params = {'min_samples_split':[8,16,24,32,40]}
min_samples_split_even_12_36 = optimize_param(X_no_race, y, rfc, params)

A min_samples_split of 16 might be good.

In [ ]:
rfc = RandomForestClassifier
params = {'min_samples_leaf':[1,2,4,8,12,16,20]}
min_samples_leaf_1 = optimize_param(X_no_race, y, rfc, params)

In [ ]:
rfc = RandomForestClassifier
params = {'max_leaf_nodes':[2,4,8,16,32]}
min_samples_leaf_1 = optimize_param(X_no_race, y, rfc, params)

In [ ]:
rfc = RandomForestClassifier(max_depth=9, min_samples_split=16)
cross_val_avg(rfc, X_no_race, y, n=5)

# No Race Data, Gradient Boost tuning

In [ ]:
gbc = GradientBoostingClassifier
params = {'max_depth':[1,3,5,7,9,11]}
max_depth_odd = optimize_param(X_no_race, y, gbc, params)

In [ ]:
gbc = GradientBoostingClassifier
params = {'learning_rate':[0.01,0.02,0.05,0.1]}
max_depth_odd = optimize_param(X_no_race, y, gbc, params)

In [ ]:
gbc = GradientBoostingClassifier
params = {'min_samples_split':[2,4,8,12,16]}
max_depth_odd = optimize_param(X_no_race, y, gbc, params)

In [ ]:
gbc = GradientBoostingClassifier(max_depth=3, learning_rate=0.01, min_samples_split=4)
cross_val_avg(gbc, X_no_race, y, n=5)

In [ ]:
gbc = GradientBoostingClassifier
params = {'min_samples_leaf':[2,4,8,12,16]}
max_depth_odd = optimize_param(X_no_race, y, gbc, params)

Changing the min_samples_leaf parameter doesn't seem to help.

# No Race Data, Support Vector Classifier

In [ ]:
params = {'C':[0.5,1,2,3,4,6]}
C_svc = optimize_param(X_no_race, y, svm.SVC, params, n=15, scale=True)

In [ ]:
params = {'gamma':[0.001,0.002,0.005,0.01,0.02]}
C_svc = optimize_param(X_no_race, y, svm.SVC, params, n=10, scale=True)

In [ ]:
params = {'gamma':[0.01,0.02,0.03,0.04,0.05]}
C_svc = optimize_param(X_no_race, y, svm.SVC, params, n=10, scale=True)

In [ ]:
parameters = {'gamma':[0.02,0.03,0.04,0.05],'C':[1,2,3,4]}
grid_search_2D(svm.SVC(), parameters, X_no_race, y, 5)

In [ ]:
parameters = {'gamma':[0.005,0.02,0.04],'C':[0.5,2,4]}
grid_search_2D(svm.SVC(), parameters, X_no_race, y, 5)

In [ ]:
cross_val_avg(svm.SVC(C=2, gamma=0.03), X_no_race, y, n=5)

# No Race Tuned Model Comparison

In [ ]:
no_race_tuned = [[1,0.5004],[6,0.5234],[300,np.nan],[3,0.5244]]
no_race_tuned_df = pd.DataFrame(data=no_race_tuned, columns=['time (s)','score'], index=['LR','RFC','GBC','SVC'])
sp = sns.scatterplot(data=no_race_tuned_df, x='time (s)', y='score', hue=no_race_tuned_df.index, s=70)
sp.legend(bbox_to_anchor= (1.2,1))
sp.set_title('No Race Data, Tuned')

# All Data, RFC and SVC

### RFC

In [ ]:
parameters = {'max_depth':[1,2,3,4],'min_samples_split':[2,4,8,12]}
grid_search_2D(RandomForestClassifier(), parameters, X_all, y, 5)

In [ ]:
parameters = {'max_depth':[3,4,5,6],'min_samples_split':[2,5,10,20]}
grid_search_2D(RandomForestClassifier(), parameters, X_all, y, 5)

In [ ]:
parameters = {'max_depth':[5,6,7,8],'min_samples_split':[2,6,12,30]}
grid_search_2D(RandomForestClassifier(), parameters, X_all, y, 5)

In [ ]:
parameters = {'max_depth':[7,8,9,10],'min_samples_split':[2,4,10,24]}
grid_search_2D(RandomForestClassifier(), parameters, X_all, y, 5)

In [ ]:
parameters = {'max_depth':[8,9,10],'min_samples_split':[2,3,5,20]}
grid_search_2D(RandomForestClassifier(), parameters, X_all, y, 5)

In [ ]:
parameters = {'max_depth':[9,10,11],'min_samples_split':[2,4,8,16]}
grid_search_2D(RandomForestClassifier(), parameters, X_all, y, 5)

In [ ]:
parameters = {'min_samples_leaf':[1,2,4,8],'min_samples_split':[2,4,8,16]}
grid_search_2D(RandomForestClassifier(max_depth=10), parameters, X_all, y, 5)

In [ ]:
parameters = {'min_samples_leaf':[4,6,8],'min_samples_split':[2,8,16,24]}
grid_search_2D(RandomForestClassifier(max_depth=10), parameters, X_all, y, 5)

In [ ]:
parameters = {'min_samples_leaf':[6],'min_samples_split':[2,4,8,16,24,32]}
grid_search_2D(RandomForestClassifier(max_depth=10), parameters, X_all, y, 5)

In [ ]:
cross_val_avg(RandomForestClassifier(max_depth=10, min_samples_leaf=6), X_all, y, n=5)

### SVC

In [ ]:
parameters = {'gamma':[0.02,0.03,0.04,0.05],'C':[1,2,3,4]}
grid_search_2D(svm.SVC(), parameters, X_all, y, 5)

In [ ]:
parameters = {'gamma':[0.005,0.01,0.015,0.02],'C':[0.5,1,2,3]}
grid_search_2D(svm.SVC(), parameters, X_all, y, 5)

In [ ]:
parameters = {'gamma':[0.001,0.002,0.004,0.005],'C':[0.5,1,2,3]}
grid_search_2D(svm.SVC(), parameters, X_all, y, 5)

In [ ]:
parameters = {'kernel':['poly','rbf','sigmoid'],'gamma':[0.001,0.005,0.01]}
grid_search_2D(svm.SVC(C=2), parameters, X_all, y, 5)

In [ ]:
parameters = {'gamma':[0.001,0.002,0.004,0.005],'C':[0.5,1,2,3]}
grid_search_2D(svm.SVC(kernel='sigmoid'), parameters, X_all, y, 5)

In [ ]:
cross_val_avg(svm.SVC(gamma=0.005, C=2), X_all, y, n=5)

# Performance Plot

Tuning hyperparameters really hasn't helped much. Feature selection seems to make a bigger impact overall.

In [ ]:
no_demo_reset_df = no_demo_df.reset_index()
no_demo_scores_df = pd.DataFrame(no_demo_reset_df.pivot(columns='index', values='score').max()).transpose()
no_demo_scores_df.index = ['no demo']
no_demo_scores_df

In [ ]:
no_race_reset_df = no_race_df.reset_index()
no_race_scores_df = pd.DataFrame(no_race_reset_df.pivot(columns='index', values='score').max()).transpose()
no_race_scores_df.index = ['no race']
no_race_scores_df

In [ ]:
all_features_reset_df = all_features_df.reset_index()
all_features_scores_df = pd.DataFrame(all_features_reset_df.pivot(columns='index', values='score').max()).transpose()
all_features_scores_df.index = ['all features']
all_features_scores_df

In [ ]:
no_race_tuned_reset_df = no_race_tuned_df.reset_index()
no_race_tuned_scores_df = pd.DataFrame(no_race_tuned_reset_df.pivot(columns='index', values='score').max()).transpose()
no_race_tuned_scores_df.index = ['no race tuned']
no_race_tuned_scores_df

In [ ]:
all_features_tuned_scores_df = pd.DataFrame(data=[[np.nan, np.nan, 0.5426, 0.5312]], columns=['GBC','LR','RFC','SVC'])
all_features_tuned_scores_df.index = ['all features tuned']
all_features_tuned_scores_df

In [ ]:
scores_df = no_demo_scores_df.append(no_race_scores_df)
scores_df = scores_df.append(no_race_tuned_scores_df)
scores_df = scores_df.append(all_features_scores_df)
scores_df = scores_df.append(all_features_tuned_scores_df)
scores_df

In [ ]:
plt.figure(figsize=(8,5))
scoreplot = sns.scatterplot(data=scores_df, s=100)
scoreplot.set(ylabel='score')

# Tuning Feature Selection

In [ ]:
def corr_limit_score(full_df, target, model, corrlist, n=5):
  df_data = []
  for cor in corrlist:
    smaller_df = drop_high_corr(full_df, target, corr_limit=cor)
    X_cols = [col for col in smaller_df.columns.to_list() if col not in non_X_cols]
    n_feats = len(X_cols)
    X = smaller_df[X_cols]
    score = cross_val_avg(model, X, y, n)
    df_data.append([cor, score, n_feats])
  return pd.DataFrame(data=df_data, columns=['Correlation Limit', 'Average Score', 'n_features'])

In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
corrlist = [0.8, 0.9, 0.95, 0.98, 0.99]
corr_scores_df = corr_limit_score(county_data_2018_df, 'Property Crime Rate', RandomForestClassifier(), corrlist)

In [ ]:
corrlist = [0.7, 0.85, 0.93, 0.97]
df = corr_limit_score(county_data_2018_df, 'Property Crime Rate', RandomForestClassifier(), corrlist)
corr_scores_df = corr_scores_df.append(df, ignore_index=True)

In [ ]:
corrlist = [0.5, 0.55, 0.6, 0.65, 0.75]
df = corr_limit_score(county_data_2018_df, 'Property Crime Rate', RandomForestClassifier(), corrlist)
corr_scores_df = corr_scores_df.append(df, ignore_index=True)

In [ ]:
corrlist = [0.3, 0.35, 0.4, 0.45]
df = corr_limit_score(county_data_2018_df, 'Property Crime Rate', RandomForestClassifier(), corrlist)
corr_scores_df = corr_scores_df.append(df, ignore_index=True)

In [ ]:
corr_scores_df.sort_values(by='Correlation Limit')

In [ ]:
plt.rc('axes', titlesize=16)
plt.rc('axes', labelsize=16)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
sns.scatterplot(data=corr_scores_df, x='Correlation Limit', y='Average Score')

In [ ]:
corrlist = [0.8, 0.9, 0.95, 0.98, 0.99]
corr_scores_svc_df = corr_limit_score(county_data_2018_df, 'Property Crime Rate', svm.SVC(), corrlist)

In [ ]:
corrlist = [0.7, 0.85, 0.93, 0.97]
df = corr_limit_score(county_data_2018_df, 'Property Crime Rate', svm.SVC(), corrlist)
corr_scores_svc_df = corr_scores_svc_df.append(df, ignore_index=True)

In [ ]:
corrlist = [0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.75]
df = corr_limit_score(county_data_2018_df, 'Property Crime Rate', svm.SVC(), corrlist)
corr_scores_svc_df = corr_scores_svc_df.append(df, ignore_index=True)

In [ ]:
sns.scatterplot(data=corr_scores_svc_df, x='Correlation Limit', y='Average Score')

In [ ]:
X_all = county_data_2018_df[all_X_cols]
cross_val_avg(RandomForestClassifier(max_depth=10, min_samples_leaf=6), X_all, y, n=15)

# Choropleths

In [ ]:
rfc_no_bias = RandomForestClassifier(n_estimators=1000, max_depth=10, min_samples_leaf=6)
#X_all = county_data_2018_df[no_race_X_cols]
rfc_no_bias.fit(county_data_2018_df[no_race_X_cols], y)
no_race_2019_X_cols = [col for col in county_data_2019_df.columns.to_list() if col not in all_race_cols and col not in non_X_cols]
X_test = county_data_2019_df[no_race_2019_X_cols]
y_test = county_data_2019_df['Property Crime Rate'].apply(lambda x: crime_level(x, q1, q2))
y_pred = rfc_no_bias.predict(X_test)
(y_test == y_pred).sum()/len(y_test)

In [ ]:
rfc_final = RandomForestClassifier(n_estimators=1000, max_depth=10, min_samples_leaf=6)
X_all = county_data_2018_df[all_X_cols]
rfc_final.fit(X_all, y)
all_2019_X_cols = [col for col in county_data_2019_df.columns.to_list() if col not in non_X_cols]
X_test = county_data_2019_df[all_2019_X_cols]
y_test = county_data_2019_df['Property Crime Rate'].apply(lambda x: crime_level(x, q1, q2))
y_pred = rfc_final.predict(X_test)
(y_test == y_pred).sum()/len(y_test)

In [ ]:
import plotly.express as px

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
county_data_2019_df['GeoFips'] = county_data_2019_df['GeoFips'].apply(lambda x: str(x) if len(str(x)) == 5 else f'0{str(x)}')

In [ ]:
fig = px.choropleth(county_data_2019_df, geojson=counties, locations='GeoFips', color='Property Crime Rate', color_continuous_scale='Bluered', scope='usa', width=1000, height=500)
fig.show()

In [ ]:
y_test_geo = pd.DataFrame()
y_test_geo['fips'] = county_data_2019_df['GeoFips']
y_test_geo['Property Crime Rate'] = y_test
fig = px.choropleth(y_test_geo, geojson=counties, locations='fips', color='Property Crime Rate', color_discrete_sequence=['orange','blue','red'], scope='usa', width=1000, height=500)
fig.show()

In [ ]:
y_pred_geo = pd.DataFrame()
y_pred_geo['fips'] = county_data_2019_df['GeoFips']
y_pred_geo['Predicted Property Crime Rate'] = y_pred
fig = px.choropleth(y_pred_geo, geojson=counties, locations='fips', color='Predicted Property Crime Rate', color_discrete_sequence=['orange','blue','red'], scope='usa', width=1000, height=500)
fig.show()

In [ ]:
y_err = pd.DataFrame()
y_err['fips'] = county_data_2019_df['GeoFips']
y_err['Property Crime Level'] = y_test
y_err['Predicted Property Crime Level'] = y_pred

In [ ]:
cld = {
    'low':1,
    'moderate':2,
    'high':3
}
def get_error(pred, actual):
    return cld[pred] - cld[actual]

In [ ]:
y_err['PCL Prediction Error'] = y_err.apply(lambda row: get_error(row['Predicted Property Crime Level'], row['Property Crime Level']), axis=1)

In [ ]:
fig = px.choropleth(y_err, geojson=counties, locations='fips', color='PCL Prediction Error', color_continuous_scale=['blue','cyan','green','orange','red'], scope='usa', width=1000, height=500)
fig.show()

# Where does it miss?

In [ ]:
y_err['State'] = county_data_2019_df['State'].map(us_state_to_abbrev)

In [ ]:
state_errors = y_err.groupby(by='State').sum()/y_err.groupby(by='State').count()

In [ ]:
state_errors = state_errors.reset_index()[['State','PCL Prediction Error']]

In [ ]:
fig = px.choropleth(state_errors, locations='State', locationmode='USA-states', color='PCL Prediction Error', color_continuous_scale='Viridis', scope='usa', width=1000, height=500)
fig.show()

In [ ]:
error_2_fips = y_err[y_err['PCL Prediction Error'] == 2]['fips']
error_minus2_fips = y_err[y_err['PCL Prediction Error'] == -2]['fips']

In [ ]:
err_2_counties_df = pd.merge(county_data_2019_df, error_2_fips, left_on='GeoFips', right_on='fips')

In [ ]:
feat_importance_df = pd.DataFrame()
feat_importance_df['feature'] = rfc_final.feature_names_in_
feat_importance_df['importance'] = rfc_final.feature_importances_
feat_import_sorted_df = feat_importance_df.sort_values(by='importance', ascending=False).reset_index(drop=True)
feat_import_sorted_df.head(10)

In [ ]:
plt.rc('axes', titlesize=16)
plt.rc('axes', labelsize=16)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.figure(figsize=(8,5))
feat_import_plot = sns.lineplot(data=feat_import_sorted_df)
plt.title('Feature Importance, final RFC')

Extensions:

Compared to other countries, does the unbiased model generalize better?

Where does the model overestimate, where does it understimate? Does anything set these counties apart?

Instead of population, use population density.